In [8]:
import pandas as pd
import os

df1 = pd.read_csv('/content/GA_TrueCar_details.csv')
df2 = pd.read_csv('/content/OR_TrueCar_details.csv')
df3 = pd.read_csv('/content/MA_TrueCar_details.csv')
df4 = pd.read_csv('/content/CA_TrueCar_details.csv')
df5 = pd.read_csv('/content/FL_TrueCar_details.csv')
df6 = pd.read_csv('/content/TX_TrueCar_details.csv')

merged_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

cleaned_df = merged_df.drop_duplicates(subset='VIN', keep='first')

cleaned_df.to_csv('TrueCars_data.csv', index=False)

total_entries = merged_df['VIN'].count()
unique_entries = cleaned_df['VIN'].count()

print(f"Total entries for VIN: {total_entries}")
print(f"Unique entries for VIN: {unique_entries}")

Total entries for VIN: 48464
Unique entries for VIN: 45920


In [9]:
from google.colab import files
files.download("TrueCars_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# import pandas as pd
# import re
# import requests
# from time import sleep

# state_mapping = {
#     'FL': 'Florida',
#     'GA': 'Georgia',
#     'CA': 'California',
#     'OR': 'Oregon',
#     'TX': 'Texas',
#     'MA': 'Massachusetts',
# }

# df = pd.read_csv('/content/TrueCars_data.csv')

# def clean_location(location):
#     cleaned_location = re.sub(r'\s\(\d+ miles away\)', '', location)

#     city_state = cleaned_location.split(',')

#     if len(city_state) == 2:
#         city = city_state[0].strip().lower().replace(" ", "+")
#         state = city_state[1].strip().upper()

#         state_name = state_mapping.get(state, state)
#         state_name = state_name.lower().replace(" ", "+")

#         return city, state_name
#     else:
#         return None, None

# df['City'], df['State'] = zip(*df['Location'].apply(clean_location))

# df = df.dropna(subset=['City', 'State'])

# unique_locations = df['City'].unique()
# print("Unique Locations:", unique_locations)

# def get_lat_lon(city, state):
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.11'}
#     url = f"https://nominatim.openstreetmap.org/search?city={city}&state={state}&country=united+states&limit=1&format=json"
#     response = requests.get(url, headers=headers).json()

#     if response:
#         lat = response[0]['lat']
#         lon = response[0]['lon']
#         return lat, lon
#     else:
#         return None, None

# latitudes = []
# longitudes = []

# for city, state in zip(df['City'], df['State']):
#     lat, lon = get_lat_lon(city, state)
#     latitudes.append(lat)
#     longitudes.append(lon)
#     sleep(1)

# df['Latitude'] = latitudes
# df['Longitude'] = longitudes

# print(df[['Location', 'Latitude', 'Longitude']])


Unique Locations: ['sanford' 'hoover' 'kennesaw' ... 'la+vernia' 'federal+heights' 'forney']


KeyboardInterrupt: 

In [11]:
# import folium

# m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# for _, row in df.iterrows():
#     if row['Latitude'] and row['Longitude']:
#         folium.Marker(
#             location=[row['Latitude'], row['Longitude']],
#             popup=folium.Popup(f"City: {row['Location']}", parse_html=True),
#             tooltip=row['Location']
#         ).add_to(m)

# m.save("interactive_map.html")

# print("Map has been saved as 'interactive_map.html'. You can open it in your browser.")


Map has been saved as 'interactive_map.html'. You can open it in your browser.


In [ ]:
import pandas as pd
import re
import requests
from time import sleep
import folium

state_mapping = {
    'FL': 'Florida',
    'GA': 'Georgia',
    'CA': 'California',
    'OR': 'Oregon',
    'TX': 'Texas',
    'MA': 'Massachusetts',
}

df = pd.read_csv('/content/TrueCars_data.csv')

def clean_location(location):

    cleaned_location = re.sub(r'\s\(\d+ miles away\)', '', location)


    city_state = cleaned_location.split(',')

    if len(city_state) == 2:
        city = city_state[0].strip().lower().replace(" ", "+")
        state = city_state[1].strip().upper()

        state_name = state_mapping.get(state, state)
        state_name = state_name.lower().replace(" ", "+")

        return city, state_name
    else:
        return None, None

df['City'], df['State'] = zip(*df['Location'].apply(lambda x: clean_location(x)))

df = df[(df['City'] != "") & (df['State'] != "")]

def get_lat_lon(city, state):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.11'}
    url = f"https://nominatim.openstreetmap.org/search?city={city}&state={state}&country=united+states&limit=1&format=json"

    try:
        # Send the API request
        response = requests.get(url, headers=headers).json()

        if response:
            lat = response[0]['lat']
            lon = response[0]['lon']
            return lat, lon
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving coordinates for {city}, {state}: {e}")
        return None, None

latitudes = []
longitudes = []

for city, state in zip(df['City'], df['State']):
    lat, lon = get_lat_lon(city, state)
    latitudes.append(lat)
    longitudes.append(lon)
    sleep(1)


df['Latitude'] = latitudes
df['Longitude'] = longitudes

df_missing_coords = df[df['Latitude'].isnull() | df['Longitude'].isnull()]
if not df_missing_coords.empty:
    print("Some locations did not return coordinates:")
    print(df_missing_coords[['Location', 'City', 'State']])

m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

for _, row in df.iterrows():
    if row['Latitude'] and row['Longitude']:  # Only add valid coordinates
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=folium.Popup(f"City: {row['Location']}", parse_html=True),
            tooltip=row['Location']
        ).add_to(m)

m.save("interactive_map.html")

print("Map has been saved as 'interactive_map.html'. You can open it in your browser.")